# Preamble

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append('../SatLib')
import satPlotTools as spt
import satbox as sb
import orbitalMechanics as om
import utils as utils

In [3]:
import numpy as np
from poliastro import constants
from poliastro.earth import Orbit
from poliastro.earth.sensors import min_and_max_ground_range, ground_range_diff_at_azimuth
from poliastro.bodies import Earth
from poliastro.maneuver import Maneuver
from poliastro.twobody.propagation import propagate
from poliastro.twobody.propagation import cowell
from poliastro.core.perturbations import J2_perturbation
from poliastro.core.propagation import func_twobody
from poliastro.util import norm
import astropy.units as u
from astropy.time import Time, TimeDelta

# Define Walker 

In [4]:
i = 60 * u.deg
t = 4
p = 2
f = 0
alt = 550 * u.km
walker = sb.Constellation.from_walker(i,t,p,f,alt)

/Users/manweichan/Desktop/MITPhD/SAS/code/.venv/lib/python3.7/site-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


# Define Ground Station

In [8]:
gs_lat = 39 * u.deg
gs_lon = 127 * u.deg
gs_h = 0 * u.m
gs = sb.GroundLoc(gs_lon, gs_lat, gs_h)

# Define RGT Drift Maneuver

In [9]:
r_drift = 650*u.km #Drift orbit at 650 km

# Calculate Burn schedule

In [10]:

schedDict = walker.gen_GOM_2_RGT_scheds(r_drift, gs)
sats2Maneuver, driftTimes, sched = walker.get_ascending_descending_per_plane(schedDict) #Assumes two satellite per plane will get there


# Propagate satellites

In [11]:
t2propagate = 1*u.day
tStep = 30 * u.s
verbose = True
walkerSim = sb.SimConstellation(walker, t2propagate, tStep, verbose = verbose)
walkerSim.propagate(select_sched_sats = sats2Maneuver, verbose=verbose)

maneuver schedule taken from satellite object
Not skipping schedule for Plane 0 Sat 0
maneuver schedule taken from satellite object
Not skipping schedule for Plane 0 Sat 1
maneuver schedule taken from satellite object
Not skipping schedule for Plane 1 Sat 2
maneuver schedule taken from satellite object
Not skipping schedule for Plane 1 Sat 3


In [27]:
#Here is the consetllation you want to plot @jace
walkerSim.constellation.planes[0].sats[0].__dict__

# This is each of the segments in the burn trajectory
walkerSim.constellation.planes[0].sats[0].cartesianRepSegments

[<CartesianRepresentation (x, y, z) in km
     [(6928.1366    ,    0.        ,    0.        ),
      (6924.39400473,   47.11553872,   81.60646647),
      (6913.16020076,   94.18008227,  163.12436366),
      (6894.41708081,  141.14227803,  244.46450039),
      (6868.13428919,  187.95005183,  325.53643017),
      (6834.26895244,  234.55023021,  406.24779391),
      (6792.76529489,  280.88814187,  486.50362638),
      (6743.55412761,  326.90718922,  566.20561061),
      (6686.55220576,  372.54838241,  645.25126678),
      (6621.66143428,  417.74982365,  723.53305446),
      (6548.76790247,  462.44612877,  800.93736541),
      (6467.7407295 ,  506.5677721 ,  877.34338251),
      (6378.43068519,  550.04033383,  952.6217686 ),
      (6280.6685545 ,  592.78362836, 1026.63314766),
      (6174.26319406,  634.71068169, 1099.22632231),
      (6058.99922747,  675.72652236, 1170.23616569),
      (5934.63430318,  715.72673598, 1239.48110036),
      (5800.89582496,  754.59572179, 1306.76005649),
    

In [28]:
#This is all the segments in one array
walkerSim.constellation.planes[0].sats[0].coordECI

<CartesianRepresentation (x, y, z) in km
    [( 6928.1366    ,     0.        ,     0.        ),
     ( 6924.39400473,    47.11553872,    81.60646647),
     ( 6913.16020076,    94.18008227,   163.12436366), ...,
     (-3160.79998242, -1270.2450554 , -5652.67625565),
     (-3026.58251635, -1446.60468503, -5709.02242767),
     (-2888.64063509, -1621.18465574, -5758.32263238)]
 (has differentials w.r.t.: 's')>